# FESDModel

FESD - Fault estimation for skeleton detection - is a suite that aims at finding faults in joints of skeletons, which are detected by human pose estimatiors.

FESDData is the sister project to this notebook, which aims at recording depth and rgb data, as well as populating the data with human poses from variing human pose estimators.

Furthermore, FESTData augments all data based on joint confidence.

FFESDModel aims to develop and evaluate a model based on the faulty and augmented joint data as well as RGBD data.

## Libraries

We need a range of libraries which are imported here. We also define some constants.

In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import os
import json
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tqdm

from dataset import FESDDataset
from utils import Frame, AugmentationParams

import cv2
import pyrealsense2 as rs

sns.set_style()

In [18]:
RECORDING_DIR = Path('H:/Recordings/')

## Data Loading

Firstly we need to import all the recordings into the notebook.


In [19]:
with open(file="Exercises.json", mode='r') as file:
  exercises_json = json.load(file)['Exercises']

with open(file="JointErrors.json", mode='r') as file:
  joint_error_json = json.load(file)

with open(file="SkeletonErrors.json", mode='r') as file:
  skeleton_error_json = json.load(file)

len(exercises_json)

13

In [25]:
dataset = FESDDataset(RECORDING_DIR, 500)

dataset.augmentation_params = AugmentationParams(flip=False, crop=False, crop_random=False, crop_pad=0, gaussian=False)
sample_frame = dataset[0]
sample_frame.show()

dataset.augmentation_params = AugmentationParams(flip=False, crop=True, crop_random=False, crop_pad=0, gaussian=False)
sample_frame = dataset[0]
sample_frame.show()

print("All Missing Joints")
np.where(np.any(sample_frame.errors==1))
print("All Wrong Joints")
print(sample_frame.poses_2d[sample_frame.errors==2])

Recordings Found: 25
Total Frames: 7500
H:\Recordings\Session_2023-02-28T22.58.23\Frames\frame_0.yml
H:\Recordings\Session_2023-02-28T22.58.23\Frames\frame_0.yml
All Missing Joints
All Wrong Joints
[[ -1.42166138 167.15560913   0.2603097 ]
 [ -1.1519165  170.9956665    0.25113821]]


In [26]:
np.where(np.any(sample_frame.errors==1))


(array([0], dtype=int64),)

## Train Model

In the following we define the training function and train a network on the training data.

In [21]:
from RD3D import train
from RD3D.model.rd3d import RD3D

import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F

from RD3D.model.rd3d import RD3D
from RD3D.data import get_loader
from RD3D.utils.func import AvgMeter, clip_gradient
from RD3D.utils.lr_scheduler import get_scheduler
from RD3D.utils.logger import setup_logger


ModuleNotFoundError: No module named 'RD3D.model.rd3d'

In [ ]:
# training
def train_salient(train_loader, model, optimizer, criterion, scheduler, epoch, opt):
    # multi-scale training  
    size_rates = [0.75, 1, 1.25]

    model.train()
    loss_record = AvgMeter()
    for i, pack in enumerate(train_loader, start=1):
        for rate in size_rates:
            optimizer.zero_grad()
            images, gts, depths = pack
            images = images.cuda()
            gts = gts.cuda()
            depths = depths.cuda()

            # multi-scale training samples
            trainsize = int(round(opt.trainsize * rate / 32) * 32)
            if rate != 1:
                images = F.upsample(images, size=(trainsize, trainsize), mode='bilinear', align_corners=True)
                images = images.unsqueeze(2)
                gts = F.upsample(gts, size=(trainsize, trainsize), mode='bilinear', align_corners=True)

                depths = F.upsample(depths, size=(trainsize, trainsize), mode='bilinear', align_corners=True)
                depths = depths.unsqueeze(2)
                images = torch.cat([images, depths], 2)

            if rate == 1:
                images = images.unsqueeze(2)
                depths = depths.unsqueeze(2)
                images = torch.cat([images, depths], 2)

            # forward
            pred_s = model(images)
            # TODO Calculate different loss based on the error label
            loss = criterion(pred_s, gts)

            loss.backward()
            clip_gradient(optimizer, opt.clip)
            optimizer.step()
            scheduler.step()
            if rate == 1:
                loss_record.update(loss.data, opt.batchsize)

        if i % 100 == 0 or i == len(train_loader):
            logger.info('Epoch [{:03d}/{:03d}], Step [{:04d}/{:04d}], Loss: {:.4f}'.
                        format(epoch, opt.epochs, i, len(train_loader),
                               loss_record.show()))